<a href="https://colab.research.google.com/github/shayesteh99/Proteome-Prediction/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import xgboost as xgb
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = "1-f4bTkNO4VI7yjDmipTySOGL18KA3-aI"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.train.csv')
gene1 = pd.read_csv('gene.train.csv')

In [ ]:
gene_train = gene1.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
id = "1Yg4vE4DCvVxQQ3lftzAf-OcFiV3K0InM"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.valid.csv')
gene2 = pd.read_csv('gene.valid.csv')

In [ ]:
gene_test = gene2.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
id = '1kHr-g60285ylV5VNJQpAnqfXWJC9pWUy'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.train.csv')
pro1 = pd.read_csv('pro.train.csv')

In [ ]:
pro_train = pro1.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
id = '1JHJEPZi5OJkMjieZT8rcSb9W8bzRPQbg'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.valid.csv')
pro2 = pd.read_csv('pro.valid.csv')

In [ ]:
pro_test = pro2.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
train_x = gene_train.to_numpy()
test_x = gene_test.to_numpy()
train_y = pro_train.to_numpy()
test_y = pro_test.to_numpy()

In [ ]:
print(train_x.shape, test_x.shape)
print(train_y.shape, test_y.shape)

(5738, 19420) (673, 19420)
(5738, 216) (673, 216)


In [ ]:
pred_test = []
pred_train = []
for i in range(test_y.shape[1]):
  print(i)
  xg_reg = xgb.XGBRegressor(n_estimators=15, nthread=4, eta=0.01, gamma=1, max_depth=7, min_child_weight=2, subsample=0.9, colsample_bytree=1, alpha=0.005, seed=27)
  xg_reg.fit(train_x,train_y[:, i])
  pred_t = xg_reg.predict(test_x)
  pred_test.append(pred_t)
  pred_tr = xg_reg.predict(train_x)
  pred_train.append(pred_tr)

In [ ]:
from sklearn.metrics import mean_squared_error,median_absolute_error,mean_absolute_error,explained_variance_score,r2_score
from math import sqrt

pt = np.asarray(pred_train).transpose()

print("mean_squared_error",mean_squared_error(train_y,pt))
print("r2_score",r2_score(train_y,pt))

mean_squared_error 0.37400906465048656
r2_score -0.23615042723231167


In [ ]:
p = np.asarray(pred_test).transpose()

print("explained_variance_score",explained_variance_score(test_y,p))
print("median_absolute_error",median_absolute_error(test_y,p))
print("r2_score",r2_score(test_y,p))
print("RMSE",sqrt(mean_squared_error(test_y,p)))
print("mean_absolute_error",mean_absolute_error(test_y,p))
print("mean_squared_error",mean_squared_error(test_y,p))

explained_variance_score 0.22534494095827237
median_absolute_error 0.4123692743770529
r2_score -0.276672265196186
RMSE 0.6301360408129831
mean_absolute_error 0.46741672238481763
mean_squared_error 0.39707142993146155


In [ ]:
test_count = len(p)
rank  = 0
for k in range(test_count):
  temp = np.append(test_y, [p[k]], axis=0)
  temp = np.array([np.subtract(i, test_y[k]) for i in temp])
  temp = np.square(temp)
  sum = temp.sum(axis = 1)
  arr = np.arange(len(sum))
  arr = np.append([sum], [arr], axis=0)
  arr = arr[ :, arr[0].argsort()]
  rank += np.where(arr[1] == test_count)[0][0]

rank /= test_count
print(rank)

18.705794947994058
